In [ ]:
!pip install faiss-cpu transformers sentence-transformers

: 

In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline

documents = [
    "Houman is 23 years old.",
    "The food is made of vegetables and meat.",
    "Tina is good at computer programming."
]

embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
doc_embeddings = embed_model.encode(documents, truncation=True)  

index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings, dtype=np.float32))

llm = pipeline("text-generation", model="tiiuae/falcon-rw-1b", device="cpu")

def rag_respond(query):
    query_embedding = embed_model.encode([query], truncation=True)  
    _, indices = index.search(np.array(query_embedding, dtype=np.float32), 1)
    best_doc = documents[indices[0][0]]

    prompt = f"Question: {query}\nRelated Knowledge: {best_doc}"
    response = llm(prompt, max_new_tokens=50, do_sample=True)[0]["generated_text"]

    return response

question = "What do you know about Houman"
answer = rag_respond(question)
print(answer)


: 

: 